In [38]:
import pandas as pd
import numpy as np

df_visits = pd.read_excel("h239g.xlsx", sheet_name=0)
df_people = pd.read_excel('h243.xlsx')
df_people = df_people[['DUPERSID', 'INSURC22']]
df = df_visits.merge(df_people, on='DUPERSID', how='left')
df.head()

,DUID,PID,DUPERSID,EVNTIDX,EVENTRN,FFEEIDX,PANEL,MPCELIG,MPCDATA,OBDATEYR,...,OBSL22X,OBWC22X,OBOT22X,OBXP22X,OBTC22X,IMPFLAG,PERWT22F,VARSTR,VARPSU,INSURC22
0,2460002,101,2460002101,2460002101008301,8,-1,24,1,1,2022,...,0.0,0.0,0.0,3.00,248.00,2,5728.309495,2082,1,6
1,2460002,101,2460002101,2460002101009001,9,-1,24,2,2,2022,...,0.0,0.0,0.0,233.22,338.00,3,5728.309495,2082,1,6
2,2460006,101,2460006101,2460006101009401,7,-1,24,1,2,2022,...,0.0,0.0,0.0,63.56,130.66,3,15648.881461,2001,4,2
3,2460006,101,2460006101,2460006101010501,8,-1,24,1,2,2022,...,0.0,0.0,0.0,366.69,719.61,3,15648.881461,2001,4,2
4,2460006,101,2460006101,2460006101010601,8,-1,24,1,2,2022,...,0.0,0.0,0.0,115.27,205.03,3,15648.881461,2001,4,2


In [41]:
MEPS_NA = [-1, -7, -8, -15, 95]
df.replace(MEPS_NA, np.nan, inplace=True)

insurance_map = {
    1: '<65 Any Private',
    2: '<65 Public Only',
    3: '<65 Uninsured',
    4: '65+ Medicare Only',
    5: '65+ Medicare + Private',
    6: '65+ Medicare + Other Public',
    7: '65+ Uninsured',
    8: '65+ No Medicare but Public/Private'
}

df['INSURANCE'] = df['INSURC22'].map(insurance_map)

df.head()

,DUID,PID,DUPERSID,EVNTIDX,EVENTRN,FFEEIDX,PANEL,MPCELIG,MPCDATA,OBDATEYR,...,OBWC22X,OBOT22X,OBXP22X,OBTC22X,IMPFLAG,PERWT22F,VARSTR,VARPSU,INSURC22,INSURANCE
0,2460002,101,2460002101,2460002101008301,8,NaN,24,1,1,2022,...,0.0,0.0,3.00,248.00,2,5728.309495,2082,1,6,65+ Medicare + Other Public
1,2460002,101,2460002101,2460002101009001,9,NaN,24,2,2,2022,...,0.0,0.0,233.22,338.00,3,5728.309495,2082,1,6,65+ Medicare + Other Public
2,2460006,101,2460006101,2460006101009401,7,NaN,24,1,2,2022,...,0.0,0.0,63.56,130.66,3,15648.881461,2001,4,2,<65 Public Only
3,2460006,101,2460006101,2460006101010501,8,NaN,24,1,2,2022,...,0.0,0.0,366.69,719.61,3,15648.881461,2001,4,2,<65 Public Only
4,2460006,101,2460006101,2460006101010601,8,NaN,24,1,2,2022,...,0.0,0.0,115.27,205.03,3,15648.881461,2001,4,2,<65 Public Only


In [42]:
cost_columns = [
    'OBTC22X',   # Total charge
    'OBSF22X',   # Out-of-pocket (family)
    'OBMR22X',   # Medicare
    'OBMD22X',   # Medicaid
    'OBPV22X',   # Private insurance
    'OBVA22X',   # VA/CHAMPVA
    'OBTR22X',   # Tricare
    'OBOF22X',   # Other federal
    'OBSL22X',   # State/local gov
    'OBWC22X',   # Worker’s comp
    'OBOT22X',   # Other insurance
]

df['COST_COVERED'] = df['OBTC22X'] - df['OBSF22X']

df.head()

,DUID,PID,DUPERSID,EVNTIDX,EVENTRN,FFEEIDX,PANEL,MPCELIG,MPCDATA,OBDATEYR,...,OBOT22X,OBXP22X,OBTC22X,IMPFLAG,PERWT22F,VARSTR,VARPSU,INSURC22,INSURANCE,COST_COVERED
0,2460002,101,2460002101,2460002101008301,8,NaN,24,1,1,2022,...,0.0,3.00,248.00,2,5728.309495,2082,1,6,65+ Medicare + Other Public,248.00
1,2460002,101,2460002101,2460002101009001,9,NaN,24,2,2,2022,...,0.0,233.22,338.00,3,5728.309495,2082,1,6,65+ Medicare + Other Public,338.00
2,2460006,101,2460006101,2460006101009401,7,NaN,24,1,2,2022,...,0.0,63.56,130.66,3,15648.881461,2001,4,2,<65 Public Only,130.66
3,2460006,101,2460006101,2460006101010501,8,NaN,24,1,2,2022,...,0.0,366.69,719.61,3,15648.881461,2001,4,2,<65 Public Only,719.61
4,2460006,101,2460006101,2460006101010601,8,NaN,24,1,2,2022,...,0.0,115.27,205.03,3,15648.881461,2001,4,2,<65 Public Only,205.03


Summary Statistics

In [43]:
def weighted_mean(x, weight):
    return (x * weight).sum() / weight.sum()


summary_weighted = df.groupby('INSURANCE').apply(
    lambda g: pd.Series({
        'Mean Total Cost': weighted_mean(g['OBTC22X'], g['PERWT22F']),
        'Mean Out-of-Pocket': weighted_mean(g['OBSF22X'], g['PERWT22F']),
        'Mean Paid By Insurer': weighted_mean(g['COST_COVERED'], g['PERWT22F'])
    })
)

print(summary_weighted)

                                    Mean Total Cost  Mean Out-of-Pocket  \
INSURANCE                                                                 
65+ Medicare + Other Public              799.119569           17.741472   
65+ Medicare + Private                   795.409308           44.723333   
65+ Medicare Only                        826.208328           39.065421   
65+ No Medicare but Public/Private       566.402505           57.944111   
65+ Uninsured                            302.161723           40.560996   
<65 Any Private                          600.402821           76.782337   
<65 Public Only                          720.451883           16.062968   
<65 Uninsured                            514.467528          113.145206   

                                    Mean Paid By Insurer  
INSURANCE                                                 
65+ Medicare + Other Public                   781.378097  
65+ Medicare + Private                        750.592383  
65+ Medicare 

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

In [ ]:
feature_cols = [
    'VISITTYPE', 'VSTCTGRY', 'DOCATLOC', 'DRSPLTY_M18',
    'MEDPTYPE_M18', 'SEEDOC_M18', 'SURGPROC', 'MEDPRESC', 'TELEHEALTHFLAG',
    'LABTEST_M18', 'XRAYS_M18', 'MRI_M18', 'EKG_M18', 'OBDATEYR', 'OBDATEMM', 'INSURANCE'
]

df[feature_cols] = df[feature_cols].fillna('Missing')
df[feature_cols] = df[feature_cols].astype(str)
df_model = df[feature_cols + ['OBTC22X', 'OBSF22X', 'COST_COVERED']]

X = df_model[feature_cols]
print(X)

targets = {
    'Total Cost': df_model['OBTC22X'],
    'Out-of-Pocket': df_model['OBSF22X'],
    'Cost Covered': df_model['COST_COVERED']
}

categorical_cols = X.select_dtypes(include='object').columns.tolist() + [
    'VISITTYPE', 'VSTCTGRY', 'DOCATLOC', 'DRSPLTY_M18',
    'MEDPTYPE_M18', 'SEEDOC_M18', 'SURGPROC', 'MEDPRESC', 'TELEHEALTHFLAG',
    'LABTEST_M18', 'XRAYS_M18', 'MRI_M18', 'EKG_M18', 'OBDATEYR', 'OBDATEMM'
]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

       VISITTYPE VSTCTGRY DOCATLOC DRSPLTY_M18 MEDPTYPE_M18 SEEDOC_M18  \
0        Missing      1.0  Missing         8.0      Missing        1.0   
1        Missing      2.0      2.0     Missing          8.0        2.0   
2        Missing      1.0  Missing        14.0      Missing        1.0   
3        Missing      2.0  Missing        14.0      Missing        1.0   
4        Missing      2.0  Missing        14.0      Missing        1.0   
...          ...      ...      ...         ...          ...        ...   
151258   Missing      1.0  Missing        14.0      Missing        1.0   
151259   Missing      5.0      2.0     Missing         15.0        2.0   
151260   Missing      1.0      2.0     Missing          2.0        2.0   
151261   Missing      2.0  Missing         6.0      Missing        1.0   
151262   Missing      2.0  Missing         6.0      Missing        1.0   

       SURGPROC MEDPRESC TELEHEALTHFLAG LABTEST_M18 XRAYS_M18  MRI_M18  \
0           2.0      1.0             

In [55]:
for target_name, y in targets.items():
    print(f"\n Prediction: {target_name}")

    # Combine features and actual target column (y is a Series with the correct column)
    df_model = pd.concat([X, y], axis=1).dropna()
    print(df_model)

    X_clean = df_model[feature_cols]
    y_clean = df_model[y.name]  # Get actual column name from Series

    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.2, random_state=42
    )

    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', LinearRegression())
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"{'Linear Regression':20} | RMSE: {rmse:8.2f} | R²: {r2:.4f}")



 Prediction: Total Cost
       VISITTYPE VSTCTGRY DOCATLOC DRSPLTY_M18 MEDPTYPE_M18 SEEDOC_M18  \
0        Missing      1.0  Missing         8.0      Missing        1.0   
1        Missing      2.0      2.0     Missing          8.0        2.0   
2        Missing      1.0  Missing        14.0      Missing        1.0   
3        Missing      2.0  Missing        14.0      Missing        1.0   
4        Missing      2.0  Missing        14.0      Missing        1.0   
...          ...      ...      ...         ...          ...        ...   
151258   Missing      1.0  Missing        14.0      Missing        1.0   
151259   Missing      5.0      2.0     Missing         15.0        2.0   
151260   Missing      1.0      2.0     Missing          2.0        2.0   
151261   Missing      2.0  Missing         6.0      Missing        1.0   
151262   Missing      2.0  Missing         6.0      Missing        1.0   

       SURGPROC MEDPRESC TELEHEALTHFLAG LABTEST_M18 XRAYS_M18  MRI_M18  \
0           